In [1]:
import pandas as pd
import pyedflib
import numpy as np
import matplotlib.pyplot as plt
from scipy import signal
from statsmodels.tsa.stattools import adfuller

patient_code = 'a0f66459'
day = 3
data_dir = f"/Users/harshil.dadlani/Desktop/UW_Courses/2021_Fall/560/Project/Datasets"

movement_init_path = f"{data_dir}/{patient_code}_mvmt_initiation.csv"
#joint_loc_path = f"{data_dir}/{patient_code}_day_{day}.csv"
ecog_path = f"{data_dir}/purged_{patient_code}_day_{day}.edf"

In [2]:
f = pyedflib.EdfReader(ecog_path) # open the file
mvmt_init_df = pd.read_csv(movement_init_path)
#joint_loc_df = pd.read_csv(joint_loc_path)

In [3]:
# # Convert to series and resample
# data = f.readSignal(4)
# periods = len(data)
# times = pd.timedelta_range(0, periods=periods, freq='L') 
# ecog_df=pd.Series(data, times)
# ecog_df = ecog_df.resample('100L').mean()

In [4]:
# inferred_freq = pd.infer_freq(pd.to_datetime(joint_loc_df['time'].head(100)))
# ecog_df = ecog_df.resample(inferred_freq).mean()


In [5]:
# Extract out timestamps
timestamps = mvmt_init_df[mvmt_init_df['day']==day]

In [6]:
def extract_timestamps(timestamps):
    indices = pd.to_timedelta(timestamps['time'].apply(lambda x : x[:-5]))
    return indices

def standardize(data):
    return (data - data.mean())/data.std()

def pre_process_ecog(series, signal_channel):
    '''
    Standardizes and removes high frequency noise components as well as line noise.
    '''
    series = standardize(series)
    f_sample = f.samplefrequency(signal_channel) # = 1000 Hz
    f_Ny = f_sample / 2. # Nyquist frequency
    f_high = 200.        # bandpass max frequency in Hz
    f_low = 1.          # bandpass min frequency in Hz
    # 1) bandpass filter
    b, a = signal.butter(4, [f_low / f_Ny, f_high / f_Ny], 'band')
    y = signal.filtfilt(b, a, series)
    # 2) bandstop filter, 60 Hz line noise and harmonics
    for mult in [1, 2, 3, 4]:
        f_stop_mid = 65. * mult
        f_stop = np.array([f_stop_mid - 2.5, f_stop_mid + 2.5])
        b, a = signal.butter(4, f_stop / f_Ny, 'bandstop')
        y = signal.filtfilt(b, a, y)
    return y
    

In [ ]:
periods = None
indices = None
for i in range(0,100):
    # Read Signal
    signal_name = f.signal_label(i)
    signal_name = str(signal_name).split(' ')[0][2:]
    
    if 'GRID' not in signal_name:
        continue
    ecog_series = f.readSignal(i)
    if periods is None:
        periods = len(ecog_series)
        times = pd.timedelta_range(0, periods=periods, freq='L') 
    ecog_series = pre_process_ecog(ecog_series,i)
    ecog_series = pd.Series(ecog_series, times)
    ecog_series = ecog_series.resample('100L').mean()
    if indices is None:
        indices = extract_timestamps(timestamps)
    ecog_series = ecog_series.loc[indices]
    if 'new timestamps' not in timestamps.columns:
        timestamps['new timestamps'] = list(ecog_series.index)
    print(signal_name)
    timestamps[signal_name] = list(ecog_series.values)

<ipython-input-7-7db3cee7278b>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  timestamps['new timestamps'] = list(ecog_series.index)
<ipython-input-7-7db3cee7278b>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  timestamps[signal_name] = list(ecog_series.values)


GRID1
GRID2
GRID3
GRID4
GRID5
GRID6
GRID7
GRID8


In [ ]:
timestamps.to_csv(f'/Users/harshil.dadlani/Desktop/UW_Courses/2021_Fall/560/Project/Datasets/Extracted_Data/extracted_{patient_code}_day_{day}.csv')

In [ ]:
timestamps